### Import Neccesory Libraries

In [14]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [15]:
df = pd.read_csv("/teamspace/studios/this_studio/Data-Science---Practices/data/final_dataset.csv")
df.head()

,Amount in rupees,Price (in rupees),location,Carpet Area in sqft,Floor,Transaction,Furnishing,facing,overlooking,Bathroom,Balcony,Ownership
0,-0.202606,-6.108617e-02,1.881061,-0.333482,-1.415544,0.547069,1.076630,1.063822,1.057353,-1.653261,-0.000319,1.341873
1,-0.056796,2.397217e-01,1.881061,-0.345214,0.378898,0.547069,-0.458287,-1.146945,-1.327164,-0.492814,-0.000319,-0.743276
2,0.052560,3.824695e-01,1.881061,-0.212259,-1.352335,0.547069,1.076630,-1.146945,-1.327164,-0.492814,-0.000319,-0.743276
3,-0.246869,1.052378e-16,1.881061,-0.320448,-1.471730,0.547069,1.076630,1.063822,1.057353,-1.653261,-1.223020,1.341873
4,0.104635,4.335362e-01,1.881061,-0.274826,-0.218078,0.547069,1.076630,1.379645,-1.201663,-0.492814,-0.000319,-1.438326


### Data splitting

In [16]:
X = df.drop(columns=['Amount in rupees'])
y = df['Amount in rupees']

### Train test split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X,y , test_size =0.2, random_state =42)